Underfitting
    - too simple to explain the data 

Optimal fit
    - not perfect, but will predict new data decently well

Overfitting
    - too good to be true (100% accuracy in a decision tree, too filtered down)
    - future data won't be accurate
    - including features that don't really apply

To avoid under/overfitting, we want to get rid of bad features

Feature Importance Measures
- VIF- tells us which overlap
- p- 
- t- 
- Coef- 

- R2- 
- MAE- 
- RMSEA- 
- Accuracy-
- Precision-
- Recall-
- F-

- pearson-
- ANOVA-
- correlation- 

In [ ]:
def import_data(path, messages=True):
  import pandas as pd
  df = pd.read_csv(path)
  if messages: print(df.shape)
  return df
        
def bin_groups(df, features=[], cutoff=0.05, replace_with='Other', messages=True):
  import pandas as pd
  if len(features) == 0: features = df.columns
  for feat in features:
    if feat in df.columns:  # Make sure they don't accidentally enter a feature name that doesn't exist
      if not pd.api.types.is_numeric_dtype(df[feat]):
        other_list = df[feat].value_counts()[df[feat].value_counts() / df.shape[0] < cutoff].index
        df.loc[df[feat].isin(other_list), feat] = replace_with
        if messages and len(other_list) > 0: print(f'{feat} has been binned by setting {other_list.values} to {replace_with}')
    else:
      if messages: print(f'{feat} not found in the DataFrame provided. No binning performed')
  return df
        
def missing_drop(df, label, row_thresh=0.7, col_thresh=0.9, drop_all=False):
  df.dropna(axis='rows', subset=[label], inplace=True)
  df.dropna(axis='columns', thresh=1, inplace=True)
  df.dropna(axis='rows', thresh=1, inplace=True)
  df.dropna(axis='columns', thresh=round(df.shape[0] * row_thresh), inplace=True)
  df.dropna(axis='rows', thresh=round(df.shape[1] * col_thresh), inplace=True)
  if drop_all: df.dropna(axis='rows', inplace=True)
  return df
  
def Xandy(df, label):
  import pandas as pd
  y = df[label]
  X = df.drop(columns=[label])
  return X, y
  
def dummy_code(X):
  import pandas as pd
  X = pd.get_dummies(X, drop_first=True)
  return X
  
def minmax(X):
  import pandas as pd
  from sklearn.preprocessing import MinMaxScaler
  X = pd.DataFrame(MinMaxScaler().fit_transform(X.copy()), columns=X.columns, index=X.index)
  return X
        
def impute_KNN(df, label, neighbors=5):
  from sklearn.impute import KNNImputer
  import pandas as pd
  X, y = Xandy(df, label)
  X = dummy_code(X.copy())
  X = minmax(X.copy())
  imp = KNNImputer(n_neighbors=neighbors, weights="uniform")
  X = pd.DataFrame(imp.fit_transform(X), columns=X.columns, index=X.index)
  return X.merge(y, left_index=True, right_index=True)
        
def fit_cv_regression(df, k, label, repeat=True, algorithm='ensemble', random_state=1, messages=True):
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  import pandas as pd
  from numpy import mean
  X, y = Xandy(df, label)
  X = dummy_code(X)
  if repeat:  cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
  else:       cv = KFold(n_splits=k, random_state=12345, shuffle=True)
  if algorithm == 'linear':
    from sklearn.linear_model import Ridge, LassoLars
    model1 = Ridge(random_state=random_state)
    model2 = LassoLars(random_state=random_state)
    score1 = mean(cross_val_score(model1, X, y, scoring='r2', cv=cv, n_jobs=-1))
    score2 = mean(cross_val_score(model2, X, y, scoring='r2', cv=cv, n_jobs=-1))
  elif algorithm == 'ensemble':
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    model1 = RandomForestRegressor(random_state=random_state)
    model2 = GradientBoostingRegressor(random_state=random_state)
    score1 = mean(cross_val_score(model1, X, y, scoring='r2', cv=cv, n_jobs=-1))
    score2 = mean(cross_val_score(model2, X, y, scoring='r2', cv=cv, n_jobs=-1))
  else:
    from sklearn.neural_network import MLPRegressor
    from sklearn.neighbors import KNeighborsRegressor
    model1 = MLPRegressor(random_state=random_state, max_iter=10000)
    model2 = KNeighborsRegressor()
    score1 = mean(cross_val_score(model1, X, y, scoring='r2', cv=cv, n_jobs=-1))
    score2 = mean(cross_val_score(model2, X, y, scoring='r2', cv=cv, n_jobs=-1))
  if messages:
    print('R2', '{: <25}'.format(type(model1).__name__), round(score1, 4))
    print('R2', '{: <25}'.format(type(model2).__name__), round(score2, 4))
  if score1 > score2: return model1.fit(X, y)
  else:               return model2.fit(X, y)

: 

In [ ]:
def bivariate(df, label, roundto=4):
    import pandas as pd, numpy as np
    from scipy import stats
        
    output_df = pd.DataFrame(columns=['unique', 'sign', 'r', 'ρ', 'τ'])
        
    for feature in df.columns:
        if feature != label:                    # Don't need the relationship fo the label with itself
            df_temp = df[[feature, label]]      # Make a version of the DataFrame with only the feature and label
            df_temp = df_temp.dropna()          # Drop missing rows from that DataFrame
            unique = df_temp[feature].nunique() # Knowing the n unique values will help determine which corr metrics to use

        r, p = stats.pearsonr(df_temp[feature], df_temp[label])
        rho, rp = stats.spearmanr(df_temp[feature], df_temp[label])
        tau, tp = stats.kendalltau(df_temp[feature], df_temp[label])
            
        if r < 0: sign = '-'  # To compare feature importance, we must convert all negative r to positive
        else:     sign = '+'  # Therefore, let's make another column to inidcate it's sign

        # Insert a record for this feature into output_df
        output_df.loc[feature] = [unique, sign, round(abs(r), roundto),  round(abs(rho), roundto), 
                                    round(abs(tau), roundto)]
    return output_df.sort_values(by=['r'], ascending=False)

    # Make a DataFrame to keep track of feature importance based on each technique we learn

df_importance = bivariate(df, 'SalePrice')
df_importance

: 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

# Create a model based on the linear algorithm with the highest fit metric
model = fit_cv_regression(df, 10, label, algorithm='linear', messages=False)

# Store the coefficients of that model in a new DataFrame
df_coef = pd.DataFrame({'Coefficients':model.coef_}, index=model.feature_names_in_)
df_coef['sign'] = '+'           # Add a column to indicate sign; default to positive
for coef in df_coef.itertuples(): # Set each coefficient to the abs() and store the sign
    if coef[1] < 0:                 # If it's negative, change to positive and store sign
        df_coef.at[coef[0], 'sign'] = '-'
        df_coef.at[coef[0], 'Coefficients'] = coef[1] * -1

df_coef.sort_values(by=['Coefficients'], inplace=True)  # Sort the results for the image
plt.figure(figsize=(4,20))                              # Set the size of the image              

# Plot the coefficients in a bar chart separately for positive vs negative with diff colors
colors = {'+': 'mediumseagreen', '-': 'lightcoral'}
df_coef['Coefficients'].plot(kind='barh', color=[colors[i] for i in df_coef['sign']])

# Create the plot legend
labels = df_coef['sign'].unique()
handles = [plt.Rectangle((0,0),1,1, color=colors[l]) for l in labels]
plt.legend(handles, labels, title="Sign")
plt.show()

# Store the coefficients in our df_importance table for later comparison
df_importance = df_importance.merge(df_coef, left_index=True, right_index=True, suffixes=('', '_coef'))
df_importance

: 

In [ ]:
model = fit_cv_regression(df, 5, label, repeat=False, algorithm='ensemble')

fi = pd.DataFrame(model.feature_importances_, columns=['FI'], index=model.feature_names_in_)
df_importance = df_importance.merge(fi, left_index=True, right_index=True)
df_importance.sort_values(by=['FI'], ascending=False, inplace=True)
df_importance

: 

In [ ]:
from sklearn.inspection import permutation_importance

X, y = Xandy(df, label)
result = permutation_importance(model, X, y, n_repeats=10, scoring="r2", random_state=1, n_jobs=-1)
df_pfi = pd.DataFrame({'PFI':result.importances_mean, 'PFI std':result.importances_std}, 
                    index=model.feature_names_in_)
df_importance = df_importance.merge(df_pfi, left_index=True, right_index=True)
df_importance.sort_values(by=['PFI'], ascending=False, inplace=True)
df_importance

: 

MULTIVARIATE FEATURE IMPORTANCE SCORES
- takes into account other aspects, while correlation features (like r [pearson correlation-normal bell curve], p [spearman-rho - normal, but integers only], t[tau - not normal]) are isolated
- more accurate than bi-variate models
- types:

    Coefficients-
        - BEST in terms of speed
        - GOOD in terms of accuracy
        - linear

    FI-
        - BETTER in terms of speed
        - BETTER in terms of accuracy
        - decision tree based model
        - if FI score is .53, it's 53% likely to show up highest in 100 (whatever number) decision trees

    PFI
        - GOOD in terms of speed
        - BEST in terms of accuracy
        - the portion of R-squared that this variable makes up

In [ ]:
def fit_cv_classification_expanded(df, label, k=10, r=5, repeat=True, random_state=1):
  import sklearn.linear_model as lm, pandas as pd, sklearn.ensemble as se, numpy as np
  from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
  from numpy import mean, std
  from sklearn import svm
  from sklearn import gaussian_process
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
  from sklearn import svm
  from sklearn.naive_bayes import CategoricalNB
  from xgboost import XGBClassifier
  from sklearn import preprocessing
  from sklearn.neural_network import MLPClassifier

  X = df.drop(columns=[label])
  y = df[label]

  if repeat:
    cv = RepeatedKFold(n_splits=k, n_repeats=r, random_state=random_state)
  else:
    cv = KFold(n_splits=k, random_state=random_state, shuffle=True)
  
  fit = {}    # Use this to store each of the fit metrics
  models = {} # Use this to store each of the models
  
  # Create the model objects
  model_log = lm.LogisticRegression(max_iter=100)
  model_logcv = lm.RidgeClassifier()
  model_sgd = lm.SGDClassifier(max_iter=1000, tol=1e-3)
  model_pa = lm.PassiveAggressiveClassifier(max_iter=1000, random_state=random_state, tol=1e-3)
  model_per = lm.Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
  model_knn = KNeighborsClassifier(n_neighbors=3)
  model_svm = svm.SVC(decision_function_shape='ovo') # Remove the parameter for two-class model
  model_nb = CategoricalNB()
  model_bag = se.BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
  model_ada = se.AdaBoostClassifier(n_estimators=100, random_state=random_state)
  model_ext = se.ExtraTreesClassifier(n_estimators=100, random_state=random_state)
  model_rf = se.RandomForestClassifier(n_estimators=10)
  model_hgb = se.HistGradientBoostingClassifier(max_iter=100)
  model_vot = se.VotingClassifier(estimators=[('lr', model_log), ('rf', model_ext), ('gnb', model_hgb)], voting='hard')
  model_gb = se.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
  estimators = [('ridge', lm.RidgeCV()), ('lasso', lm.LassoCV(random_state=random_state)), ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))]
  final_estimator = se.GradientBoostingRegressor(n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1, random_state=random_state)
  model_st = se.StackingRegressor(estimators=estimators, final_estimator=final_estimator)
  model_xgb = XGBClassifier()
  model_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=random_state)

  # Fit a cross-validated R squared score and add it to the dict
  fit['Logistic'] = mean(cross_val_score(model_log, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Ridge'] = mean(cross_val_score(model_logcv, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SGD'] = mean(cross_val_score(model_sgd, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['PassiveAggressive'] = mean(cross_val_score(model_pa, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Perceptron'] = mean(cross_val_score(model_per, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['KNN'] = mean(cross_val_score(model_knn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['SVM'] = mean(cross_val_score(model_svm, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NaiveBayes'] = mean(cross_val_score(model_nb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Bagging'] = mean(cross_val_score(model_bag, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['AdaBoost'] = mean(cross_val_score(model_ada, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['ExtraTrees'] = mean(cross_val_score(model_ext, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['RandomForest'] = mean(cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['HistGradient'] = mean(cross_val_score(model_hgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['Voting'] = mean(cross_val_score(model_vot, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['GradBoost'] = mean(cross_val_score(model_gb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  fit['NeuralN'] = mean(cross_val_score(model_nn, X, y, scoring='accuracy', cv=cv, n_jobs=-1))
  
  # XGBoost needs to LabelEncode the y before fitting the model
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder().fit(y)
  y_encoded = le.transform(y.copy())
  fit['XGBoost'] = mean(cross_val_score(model_xgb, X, y_encoded, scoring='accuracy', cv=cv, n_jobs=-1))

  # Add the model to another dictionary; make sure the keys have the same names as the list above
  models['Logistic'] = model_log
  models['Ridge'] = model_logcv
  models['SGD'] = model_sgd
  models['PassiveAggressive'] = model_pa
  models['Perceptron'] = model_per
  models['KNN'] = model_knn
  models['SVM'] = model_svm
  models['NaiveBayes'] = model_nb
  models['Bagging'] = model_bag
  models['AdaBoost'] = model_ada
  models['ExtraTrees'] = model_ext
  models['RandomForest'] = model_rf
  models['HistGradient'] = model_hgb
  models['Voting'] = model_vot
  models['GradBoost'] = model_gb
  models['XGBoost'] = model_xgb
  models['NeuralN'] = model_nn

  # Add the fit dictionary to a new DataFrame, sort, extract the top row, use it to retrieve the model object from the models dictionary
  df_fit = pd.DataFrame({'Accuracy':fit})
  df_fit.sort_values(by=['Accuracy'], ascending=False, inplace=True)
  best_model = df_fit.index[0]
  print(df_fit)

  return models[best_model].fit(X, y)

: 

In [ ]:
# Setting the label here since it is used in multiple function calls
label = 'SalePrice'

# Import the data
df = import_data('housing.csv', messages=False)

# Clean/prepare the data
df = bin_groups(df, messages=False)
df = missing_drop(df, label)
df = impute_KNN(df, label)
df.head()

# Fit the model based on the algorithm with the best fit metrics
print(f'Model fit before feature selection:')
model = fit_cv_regression(df, 5, label)

# Select the best features based on the model that was stored above
df_reduced = select_features(df.copy(), label, model)

# Print out the number of features as a sanity check
print(f'\nShape after feature selection:\t{df_reduced.shape}\n')

# Retrain the model with the reduced feature set
print(f'Model fit after feature selection:')
model = fit_cv_regression(df_reduced, 5, label)

: 